<a href="https://www.kaggle.com/code/maxigaarp/c3-rpostgres?scriptVersionId=136065017" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Rpostgres setup

Para iniciar el servidor virtual, instalar la base de datos postgres, y descargar los datos e importarlos, debe correr los siguientes bloques.

In [85]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [1]:
install.packages("RPostgres")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library("DBI")
library("RPostgreSQL")
library("RODBC")



In [3]:
system("sudo apt install -y postgresql postgresql-contrib &>log", intern=T)

[1] "Reading package lists..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
  [2] "Building dependency tree..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
  [3] "Reading state information..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
  [4] "The following additional packages will be installed:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
  [5] "  cron libpopt0 libpq-dev libpq5 libxslt1.1 logrotate postgresql-12"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
  [6] "  postgresql-client-12 postgresql-client-common postgresql-common ssl-cert"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
  [7] "  sysstat"                                                                                                             

In [4]:
system("service postgresql start", intern = T)

[1] " * Starting PostgreSQL 12 database server"
[2] "   ...done."

In [5]:
system("sudo -u postgres psql -c \"CREATE USER root WITH SUPERUSER\"", intern=T)


[1] "CREATE ROLE"

In [6]:
system("sudo -u postgres createdb clase", intern=T)

character(0)

In [7]:
system("sudo -u postgres psql clase < /kaggle/input/sql-guia/guia2.sql", intern=T)

[1] "SET"                      "SET"                     
 [3] "SET"                      "SET"                     
 [5] "SET"                      " set_config "            
 [7] "------------"             " "                       
 [9] "(1 row)"                  ""                        
[11] "SET"                      "SET"                     
[13] "SET"                      "SET"                     
[15] "CREATE SCHEMA"            "SET"                     
[17] "SET"                      "CREATE TABLE"            
[19] "ALTER TABLE"              "CREATE TABLE"            
[21] "ALTER TABLE"              "CREATE TABLE"            
[23] "ALTER TABLE"              "COPY 14676"              
[25] "COPY 250"                 "COPY 17229"              
[27] "ALTER TABLE"              "ALTER TABLE"             
[29] "ALTER TABLE"              "ALTER TABLE"             
[31] "ALTER TABLE"              "GRANT"                   
[33] "GRANT"                    "GRANT"                   
[35] "GRANT"                    "ALTER DEFAULT PRIVILEGES"
[37] "ALTER DEFAULT PRIVILEGES"


Se presentan los datos de películas extraídos de IMDb. El esquema de los datos es el siguiente:

* $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\underline{\text{anho}}}, \color{blue}{\text{calificacion}})$ 
* $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\text{genero}})$
* $\color{green}{\textbf{personaje}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}},  \color{blue}{\underline{\text{a_nombre}}}, \color{blue}{\underline{\text{personaje}}})$

La tabla $\color{green}{\textbf{personaje}}$ usa llaves foráneas que hacen referencia a las tablas de $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{a_nombre}}})$ y $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}})$.



# Connect

In [50]:
con <- dbConnect(RPostgres::Postgres(),dbname = 'clase', user="root", password = 'PASSWORD', options="-c search_path=guia")


In [9]:
con

<PqConnection> clase@/var/run/postgresql:5432

In [ ]:
dbDisconnect(con)

In [ ]:
con

In [10]:
dbListTables(con) #lista de tablas

[1] "personaje" "actor"     "pelicula"

In [11]:
dbListFields(con, "personaje")

[1] "a_nombre"  "p_nombre"  "p_anho"    "personaje"

In [12]:
dbReadTable(con, "pelicula")

nombre,anho,calificacion
<chr>,<int>,<dbl>
The Shawshank Redemption,1994,9.2
The Godfather,1972,9.2
The Godfather: Part II,1974,9.0
The Dark Knight,2008,8.9
12 Angry Men,1957,8.9
Schindler's List,1993,8.9
Pulp Fiction,1994,8.9
The Lord of the Rings: The Return of the King,2003,8.9
"Il buono, il brutto, il cattivo",1966,8.9


Ejecute la siguiente consulta para probar que todo ande bien:

In [13]:
head(dbReadTable(con, "actor"),10)

,nombre,genero
,<chr>,<chr>
1,"Aames, Angela",F
2,"Aames, Marlene",F
3,"Abad, Regina",F
4,"Abbott, Diahnne",F
5,"Abbott, Dorothy",F
6,"Abbott, Jane (II)",F
7,"Abbott, Jennifer M.",F
8,"Aberlé, Viola",F
9,"Abramova, Natalya (I)",F


## Conectarse a bases de datos publicas:

https://rnacentral.org/help/public-database#:~:text=The%20public%20Postgres%20database%20allows,the%20instructions%20available%20on%20GitHub.

In [14]:
con <- dbConnect(RPostgres::Postgres(),dbname = 'pfmegrnargs',host="hh-pgsql-public.ebi.ac.uk",port=5432, user="reader", password = 'NWDMCE5xdipIjRrp', options="-c search_path=guia")


In [15]:
"select * from table; drop table"


dbListTables(con) #lista de tablas

character(0)

# Cargar base de datos a Rpostgres

Creemos nueva conexión a una base de datos limpia.

In [16]:
system("sudo -u postgres createdb EjemploCarga", intern=T)

character(0)

In [17]:
con2 <- dbConnect(RPostgres::Postgres(),dbname = 'EjemploCarga', user="root", password = 'PASSWORD')


In [18]:
dbListTables(con2)

character(0)

In [19]:
con2

<PqConnection> EjemploCarga@/var/run/postgresql:5432

In [20]:
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


In [22]:
dbWriteTable(con2, "cars", mtcars)

ERROR: Error: Table cars exists in database, and both overwrite and append are FALSE


In [24]:
dbListTables(con2)

[1] "cars"

In [25]:
dbReadTable(con2, 'cars')

mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


In [27]:
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


# Consultas

In [28]:
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


In [29]:
rownames(mtcars)

[1] "Mazda RX4"           "Mazda RX4 Wag"       "Datsun 710"         
 [4] "Hornet 4 Drive"      "Hornet Sportabout"   "Valiant"            
 [7] "Duster 360"          "Merc 240D"           "Merc 230"           
[10] "Merc 280"            "Merc 280C"           "Merc 450SE"         
[13] "Merc 450SL"          "Merc 450SLC"         "Cadillac Fleetwood" 
[16] "Lincoln Continental" "Chrysler Imperial"   "Fiat 128"           
[19] "Honda Civic"         "Toyota Corolla"      "Toyota Corona"      
[22] "Dodge Challenger"    "AMC Javelin"         "Camaro Z28"         
[25] "Pontiac Firebird"    "Fiat X1-9"           "Porsche 914-2"      
[28] "Lotus Europa"        "Ford Pantera L"      "Ferrari Dino"       
[31] "Maserati Bora"       "Volvo 142E"

In [30]:
mtcars$name=rownames(mtcars)

In [31]:
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,Valiant
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,Duster 360
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,Merc 240D
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,Merc 230


In [32]:
rownames(mtcars) <- 1:nrow(mtcars)


In [33]:
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
2,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
3,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
4,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
5,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
6,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,Valiant
7,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,Duster 360
8,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,Merc 240D
9,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,Merc 230


In [34]:
dbExecute(con2, "DROP TABLE cars")

[1] 0

In [39]:
dbListTables(con2)

[1] "cars"  "cars2"

In [36]:
dbWriteTable(con2, "cars", mtcars)

In [43]:
dbReadTable(con2,"cars2")

mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,name
<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<chr>
21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,Valiant
14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,Duster 360
24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,Merc 240D
22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,Merc 230


In [38]:
dbExecute(con2, "CREATE TABLE cars2(mpg FLOAT,cyl INT,disp FLOAT, hp INT, drat FLOAT, wt FLOAT, qsec FLOAT, vs INT,am INT, gear INT, carb INT, name VARCHAR(30))")

[1] 0

In [42]:
dbAppendTable(con2, "cars2", mtcars)

[1] 32

In [46]:
dbGetQuery(con2, "select * from cars2 where cyl=6;")

mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,name
<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<chr>
21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,Valiant
19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4,Merc 280
17.8,6,167.6,123,3.92,3.440,18.90,1,0,4,4,Merc 280C
19.7,6,145.0,175,3.62,2.770,15.50,0,1,5,6,Ferrari Dino


In [ ]:
dbAppendTable(con2, "cars", mtcars)

In [ ]:
dbReadTable(con2, "cars")

In [47]:

dbGetQuery(con2, "SELECT * FROM information_schema.columns WHERE TABLE_NAME = 'cars';")

table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,⋯,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
EjemploCarga,public,cars,mpg,1,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES
EjemploCarga,public,cars,cyl,2,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES
EjemploCarga,public,cars,disp,3,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES
EjemploCarga,public,cars,hp,4,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES
EjemploCarga,public,cars,drat,5,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES
EjemploCarga,public,cars,wt,6,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES
EjemploCarga,public,cars,qsec,7,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES
EjemploCarga,public,cars,vs,8,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES
EjemploCarga,public,cars,am,9,NA,YES,double precision,NA,NA,⋯,NO,NA,NA,NA,NA,NA,NO,NEVER,NA,YES


---
Volvamos a base de datos de imdb

In [51]:
dbListTables(con)

[1] "personaje" "actor"     "pelicula"

In [58]:
head(dbGetQuery(con,"SELECT * FROM pelicula"),10)

,nombre,anho,calificacion
,<chr>,<int>,<dbl>
1,The Shawshank Redemption,1994,9.2
2,The Godfather,1972,9.2
3,The Godfather: Part II,1974,9.0
4,The Dark Knight,2008,8.9
5,12 Angry Men,1957,8.9
6,Schindler's List,1993,8.9
7,Pulp Fiction,1994,8.9
8,The Lord of the Rings: The Return of the King,2003,8.9
9,"Il buono, il brutto, il cattivo",1966,8.9


In [86]:
dbGetQuery(con,"SELECT * FROM actor") %>% filter(genero=='F') 

Warning message in result_create(conn@ptr, statement, immediate):
“Closing open result set, cancelling previous query”


nombre,genero
<chr>,<chr>
"Aames, Angela",F
"Aames, Marlene",F
"Abad, Regina",F
"Abbott, Diahnne",F
"Abbott, Dorothy",F
"Abbott, Jane (II)",F
"Abbott, Jennifer M.",F
"Aberlé, Viola",F
"Abramova, Natalya (I)",F


In [82]:
query <- dbSendQuery(con, "select  * from actor")

In [72]:
sum(tabla$genero=='F')

[1] 10

In [81]:
query

<PqResult>
EXPIRED

In [83]:
contador<- 0

while (!dbHasCompleted(query)){
    tabla = dbFetch(query, 10)
    contador <- contador + sum(tabla$genero=="F")
}
contador


[1] 3748


## Pregunta 1
Las películas de los 80s, ordenadas por calificación de mayor a menor.

In [ ]:
select <columnas>
from <tablas>
where <condiciones>
group by
order by
limit 
having


In [87]:
dbGetQuery(con, 
"SELECT * FROM pelicula
WHERE anho BETWEEN 1980 AND 1989
ORDER BY calificacion DESC;")


nombre,anho,calificacion
<chr>,<int>,<dbl>
Star Wars: Episode V - The Empire Strikes Back,1980,8.7
Raiders of the Lost Ark,1981,8.5
Back to the Future,1985,8.5
Nuovo Cinema Paradiso,1988,8.4
The Shining,1980,8.4
Hotaru no haka,1988,8.4
Aliens,1986,8.4
Once Upon a Time in America,1984,8.4
Das Boot,1981,8.3


In [ ]:
dbGetQuery(con, 
"SELECT *")

## Pregunta 2
Los nombres de los personajes que ha interpretado su actriz/actor favorito en los datos, ordenados por año.

In [105]:
dbGetQuery(con, "select * from personaje where a_nombre='DiCaprio, Leonardo' ORDER BY p_anho;")

dbGetQuery(con, "select * from personaje where a_nombre='Streep, Meryl' ORDER BY p_anho")



a_nombre,p_nombre,p_anho,personaje
<chr>,<chr>,<int>,<chr>
"DiCaprio, Leonardo",Catch Me If You Can,2002,Frank Abagnale Jr.
"DiCaprio, Leonardo",The Departed,2006,Billy
"DiCaprio, Leonardo",Shutter Island,2010,Teddy Daniels
"DiCaprio, Leonardo",Inception,2010,Cobb
"DiCaprio, Leonardo",Django Unchained,2012,Calvin Candie
"DiCaprio, Leonardo",The Wolf of Wall Street,2013,Jordan Belfort
"DiCaprio, Leonardo",The Revenant,2015,Hugh Glass


a_nombre,p_nombre,p_anho,personaje
<chr>,<chr>,<int>,<chr>
"Streep, Meryl",The Deer Hunter,1978,Linda


In [104]:
dbGetQuery(con, "select * from actor where nombre like '%Meryl%'")

nombre,genero
<chr>,<chr>
"Streep, Meryl",F


## Pregunta 3
Las películas en las que participó su actriz/actor favorito en los datos, ordenadas por calificación de mayor a menor.

In [107]:
dbGetQuery(con, "select * from personaje limit 10")

a_nombre,p_nombre,p_anho,personaje
<chr>,<chr>,<int>,<chr>
"Aames, Angela",Scarface,1983,Woman at Babylon Club
"Aames, Marlene",The Best Years of Our Lives,1946,Luella Parrish
"Abad, Regina",Amores perros,2000,Jimena
"Abbott, Diahnne",Taxi Driver,1976,Concession Girl
"Abbott, Dorothy",The Apartment,1960,Office Worker
"Abbott, Jane (II)",The Lord of the Rings: The Return of the King,2003,Coronation Elf
"Abbott, Jennifer M.",Casino,1995,Cashier
"Aberlé, Viola",Fanny och Alexander,1982,Japanese Woman - Jacobis hus
"Abramova, Natalya (I)",Stalker,1979,"Marta, doch Stalkera"


In [109]:
dbGetQuery(con, 
"SELECT p_nombre, p_anho, calificacion
FROM pelicula as P, personaje as J
WHERE P.nombre = p_nombre
  AND P.anho = p_anho
  AND a_nombre='DiCaprio, Leonardo'
  ORDER BY P.calificacion DESC;")

p_nombre,p_anho,calificacion
<chr>,<int>,<dbl>
Inception,2010,8.7
The Departed,2006,8.5
Django Unchained,2012,8.4
The Wolf of Wall Street,2013,8.2
Shutter Island,2010,8.1
The Revenant,2015,8.0
Catch Me If You Can,2002,8.0


## Pregunta 4
Los nombres de los personajes interpretados por mujeres, en películas de los 90s, con calificación mayor a 8,7.

In [112]:
dbGetQuery(con, 
"SELECT * from actor limit 10;")

nombre,genero
<chr>,<chr>
"Aames, Angela",F
"Aames, Marlene",F
"Abad, Regina",F
"Abbott, Diahnne",F
"Abbott, Dorothy",F
"Abbott, Jane (II)",F
"Abbott, Jennifer M.",F
"Aberlé, Viola",F
"Abramova, Natalya (I)",F


In [118]:
dbGetQuery(con, 
"SELECT personaje
FROM pelicula 
JOIN personaje ON pelicula.nombre=p_nombre
JOIN actor ON a_nombre=actor.nombre
WHERE genero='F' AND 
p_anho BETWEEN 1990 AND 1999 AND
calificacion>=8.7;")

personaje
<chr>
Winston Wolfe's Girlfriend At Party
Fourth Man
Jody
Young Henry's Sister #1
Herself - Schindler Mourner
Ghetto Girl
Marie #1
"Susan, Cosmetics Dealer"
Hippie


## Pregunta 5
Las películas de la saga "The Lord of The Rings" (usando el prefijo de su nombre) ordenadas por calificación y por año.

In [120]:
dbGetQuery(con, 
"SELECT * FROM pelicula
WHERE nombre LIKE 'The Lord of the Rings%'
ORDER BY calificacion, anho;")

nombre,anho,calificacion
<chr>,<int>,<dbl>
The Lord of the Rings: The Two Towers,2002,8.7
The Lord of the Rings: The Fellowship of the Ring,2001,8.8
The Lord of the Rings: The Return of the King,2003,8.9


## Pregunta 6
Los nombres de los actores que interpretan más de un personaje en la misma película.

## Pregunta 7
Las películas en que actúan juntos Uma Thurman y Samuel L. Jackson.

In [ ]:
VARIABLE_PERSONAJE <- 'SAMUEL JACKSON'
dbGetQuery(con, 
"SELECT p_nombre, p_anho FROM personaje
WHERE a_nombre='Thurman, Uma'
INTERSECT
SELECT p_nombre, p_anho FROM personaje
WHERE a_nombre='VARIABLE_PERSONAJE';")

## Pregunta 8
Las películas en que actúa Uma Thurman y **no** Samuel L. Jackson.

## Pregunta 9
La película con calificación más alta.

# Consultas por lotes 

## Pregunta 10

Los pares de actores que aparecen juntos en más de una película. Cada par debe aparecer una sola vez, es decir, si $(A,B)$ aparece, no debe aparecer $(B,A)$, pues es el mismo par. Tampoco se deben incluir pares de la forma $(A,A)$.

In [ ]:
query <- dbSendQuery(con, 
"SELECT DISTINCT p1.a_nombre,p2.a_nombre
FROM personaje p1,personaje p2,personaje p3,personaje p4
WHERE p1.p_nombre = p2.p_nombre
  AND p1.p_anho = p2.p_anho
  AND p1.a_nombre < p2.a_nombre
  AND p3.p_nombre = p4.p_nombre
  AND p3.p_anho = p4.p_anho
  AND p1.a_nombre = p3.a_nombre
  AND p2.a_nombre = p4.a_nombre
  AND p1.p_nombre <> p3.p_nombre
  AND p1.p_anho <> p3.p_anho;")

In [ ]:
dbFetch(query, 10)

In [ ]:
dbFetch(query, 10)

In [ ]:
while (!dbHasCompleted(query)) {
  df <- dbFetch(query, n = 500)
  print(nrow(df))
}

In [ ]:
dbClearResult(query)

In [ ]:
query

# Consultas parametrizadas

## Pregunta 11
Los nombres de los personajes interpretados por mujeres, en peliculas del año X, con calificación mayor a Y.

In [121]:
query <- dbSendQuery(con, 
"SELECT personaje
FROM pelicula P, actor A, personaje J
WHERE a_nombre=A.nombre
  AND P.nombre = p_nombre
  AND P.anho = p_anho
  AND p_anho = $1
  AND P.calificacion > $2
  AND A.genero = 'F';")

In [123]:
dbFetch(query,10)

ERROR: Error: Query needs to be bound before fetching


In [128]:
dbBind(query, list(1990, 8))

In [129]:
dbFetch(query,10)

personaje
<chr>
Young Henry's Sister #1
Marie #1
Nicky Eyes' Girlfriend
Karen Hill
Carbone's Girlfriend
Gangster's Girlfriend
Joe Buddha's Wife
Vito's Girlfriend
Lisa


#Agrupando

¿Cual seria la base de datos que contiene todos los cruces de informacion?

In [ ]:
dbGetQuery(con, "
SELECT pelicula.nombre as pelicula_nombre,pelicula.calificacion, anho, a_nombre as actor_nombre, personaje, genero as actor_genero
FROM pelicula JOIN personaje ON
pelicula.nombre= personaje.p_nombre 
AND   pelicula.anho=personaje.p_anho JOIN actor
ON personaje.a_nombre=actor.nombre
")

In [ ]:
imdb <- dbGetQuery(con, "
SELECT pelicula.nombre as pelicula_nombre,pelicula.calificacion, anho, a_nombre as actor_nombre, personaje, genero as actor_genero
FROM pelicula, personaje, actor
WHERE pelicula.nombre= personaje.p_nombre 
AND   pelicula.anho=personaje.p_anho
AND   personaje.a_nombre=actor.nombre
")

## Pregunta 12

Contar el numero de actores de cada pelicula

In [ ]:
dbGetQuery(con, "
SELECT pelicula.nombre as pelicula_nombre, anho, count(actor.nombre)
FROM pelicula, personaje, actor
WHERE pelicula.nombre= personaje.p_nombre 
AND   pelicula.anho=personaje.p_anho
AND   personaje.a_nombre=actor.nombre
GROUP BY  pelicula.nombre, pelicula.anho
")

¿como lograriamos esto en R?

In [ ]:
library(tidyverse)

In [ ]:
imdb %>% group_by(pelicula_nombre, anho)%>% # esto me genera una serie de grupos segun los atributos 
         summarise(count=n())

In [ ]:
imdb %>% group_by(pelicula_nombre, anho)%>% # esto me genera una serie de grupos segun los atributos 
         summarise(count=n_distinct(actor_nombre))

## Pregunta 13

Contar en numero de personajes distintos para cada actor:

##Pregunta 14
Obtener nombre de las actrices con un promedio de peliculas superior a 8.5

In [ ]:
dbGetQuery(con,"
SELECT actor.nombre, AVG(pelicula.calificacion), actor.genero
FROM personaje 
JOIN pelicula ON
pelicula.nombre= personaje.p_nombre 
AND  pelicula.anho=personaje.p_anho 
JOIN actor ON
personaje.a_nombre=actor.nombre
WHERE actor.genero = 'F'
GROUP BY actor.nombre
HAVING AVG(pelicula.calificacion)>8.5
")

In [ ]:
imdb %>% filter(actor_genero=="F") %>% 
         group_by(actor_nombre) %>%
         mutate(calificacion_prom=mean(calificacion),
                   genero=actor_genero) %>%
         filter(calificacion_prom>8.5) 


#Riiid DB




In [ ]:
install.packages(RSQLite)

In [ ]:
system("gdown https://drive.google.com/uc?id=1-4yIBsv8Z1aAz8kMRbMC4OXEe7kVZ_td")
system("gdown https://drive.google.com/uc?id=1-OqMYeOe-f4u1VbU1H0sCSGSK5PsAh30")
system("gdown https://drive.google.com/uc?id=1-IbkHHJyxxnhlKb7dCbw5nOq_imr4qrg")

In [ ]:
conn <- dbConnect(RSQLite::SQLite(), "riiid.db")
dbListTables(conn)

In [ ]:
dbWriteTable(conn , name = "train", 
                  value = "/content/train.csv",
                  row.names = FALSE, header = T, sep=',',
                  colClasses='character')


In [ ]:
dbWriteTable(conn , name = "questions", 
                  value = "/content/questions.csv",
                  row.names = FALSE, header = T, sep=',',
                  colClasses='character')


In [ ]:
dbWriteTable(conn , name = "lectures", 
                  value = "/content/lectures.csv",
                  row.names = FALSE, header = T, sep=',',
                  colClasses='character')

In [ ]:
dbListTables(conn)

In [ ]:
dbGetQuery(conn,"select * from train limit 100")

In [ ]:
dbGetQuery(conn, "SELECT
    user_id,
    content_id,
    answered_correctly, 
    LAG(answered_correctly) OVER (
        PARTITION BY user_id 
        ORDER BY (timestamp)) as prev_answered_correctly,
    AVG(answered_correctly) OVER (PARTITION BY user_id) 
        AS mean_answered,
    count(answered_correctly) OVER (PARTITION BY user_id)
FROM
    train
    Limit 1000;") # esto toma aproximadamente 10 min

In [ ]:
install.packages("tictoc")
library(tictoc)

In [ ]:
tic()
query <- dbSendQuery(conn, "
SELECT
    user_id,
    content_id,
    answered_correctly, 
    LAG(answered_correctly) OVER (
        PARTITION BY user_id 
        ORDER BY (timestamp)) as prev_answered_correctly,
    AVG(answered_correctly) OVER (PARTITION BY user_id) 
        AS mean_answered,
    count(answered_correctly) OVER (PARTITION BY user_id)
FROM
  train")
toc()

In [ ]:
tic()
query <- dbSendQuery(conn, "SELECT * FROM train;")
toc()

In [ ]:
dbFetch(query,n=100)

In [ ]:
DBI::dbHasCompleted(query)

In [ ]:
DBI::dbDisconnect(conn)